In [ ]:
import os 
from PIL import Image

Convert Bmp to png

In [ ]:
# Define the directory containing the BMP images
def convert_bmp_to_png(input_directory,output_directory):

    for filename in os.listdir(input_directory):
        if filename.lower().endswith('.bmp'):
            # Construct full file path
            bmp_path = os.path.join(input_directory, filename)
            # Open the BMP image
            with Image.open(bmp_path) as bmp_image:
                # Convert the file extension to .png
                png_filename = os.path.splitext(filename)[0] + '.png'
                png_path = os.path.join(output_directory, png_filename)
                # Save the image as PNG
                bmp_image.save(png_path, 'PNG')
                print(f"Converted {filename} to {png_filename}")

print("Conversion completed.")


In [ ]:
input_dir = r'C:\Users\user\Desktop\medical_image_analysis\Covid\images'
output_dir = r'C:\Users\user\Desktop\medical_image_analysis\Covid_png_format\covid_images'

convert_bmp_to_png(input_dir,output_dir)

In [ ]:
input_dir_covid_masks = r'C:\Users\user\Desktop\medical_image_analysis\Covid\masks'
output_dir_covid_masks = r'C:\Users\user\Desktop\medical_image_analysis\Covid_png_format\covid_masks'

convert_bmp_to_png(input_dir_covid_masks, output_dir_covid_masks)

In [ ]:
input_dir_non_covid = r'C:\Users\user\Desktop\medical_image_analysis\Non_Covid\images'
output_dir_non_covid = r'C:\Users\user\Desktop\medical_image_analysis\Non_covid_png_format\non_covid_images'

convert_bmp_to_png(input_dir_non_covid,output_dir_non_covid)

In [ ]:
input_dir_non_covid_masks = r'C:\Users\user\Desktop\medical_image_analysis\Non_Covid\masks'
output_dir_non_covid_masks = r'C:\Users\user\Desktop\medical_image_analysis\Non_covid_png_format\non_covid_masks'

convert_bmp_to_png(input_dir_non_covid_masks,output_dir_non_covid_masks)

importing necessary libraries

In [1]:
import os
from glob import glob
import numpy as np
import torch
# from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
# from albumentations.pytorch import ToTensorV2
import cv2

In [7]:
from keras.utils import Sequence

In [2]:
augmentation_pipeline = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=20, p=0.5),
        A.RandomResizedCrop(height=128, width=128, scale=(0.8, 1.0), p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        # ToTensorV2()
])

In [3]:
mask_transform_pipeline = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=20, p=0.5),
        A.RandomResizedCrop(height=128, width=128, scale=(0.8, 1.0), p=0.5)
        # ToTensorV2()
        # A.Resize(height=256, width=256)
    ])

In [4]:
val_image_augmentation_pipeline = A.Compose([
    A.Resize(height=128, width=128), 
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    # ToTensorV2()
])

In [5]:
val_mask_augmentation_pipeline = A.Compose([
    A.Resize(height=128, width=128),
    
])

Augmentation method 

In [8]:
class AugmentedDataset(Sequence):
    def __init__(self, image_paths, mask_paths, image_size=(128,128),augment=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.image_size = image_size
        self.augment = augment

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        img_path = self.image_paths[index]
        mask_path = self.mask_paths[index]

        # for img_path, mask_path in zip(batch_image_paths, batch_mask_paths):
        image = cv2.imread(img_path)
        image = cv2.resize(image, self.image_size)
        image = np.array(image)
        mask = cv2.imread(mask_path)
        mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
        # mask = mask.resize(self.image_size)
        mask = cv2.resize(mask, self.image_size)
        mask = np.reshape(mask,mask.shape+(1,))

        mask = np.array(mask)
        # image = np.array(cv2.imre(img_path).convert("RGB").resize(self.image_size))
        # mask = np.array(Image.open(mask_path).resize(self.image_size))

        if self.augment and os.path.basename(img_path).startswith('Non'):
            augmented = augmentation_pipeline(image=image)
            image = augmented['image']
            augmented = mask_transform_pipeline(image=mask)
            mask = augmented['image']
        else:
            if self.augment and os.path.basename(img_path).startswith('Val'):
            # Apply resizing and normalization to validation data
                augmented = val_image_augmentation_pipeline(image=image)
                image = augmented['image']
                augmented = val_mask_augmentation_pipeline(image=mask)
                mask = augmented['image']

        # img_arr = np.array(image)
        # mask_arr = np.array(mask)

        # img_path.append(image)
        # mask_path.append(mask)

        # images = np.array(images)
        # masks = np.array(masks)
        # print(type(image))
        # print(type(mask))
        return image, mask
            
    
     

In [9]:

image_folder = r'C:\Users\user\Desktop\medical_image_analysis\dataset\train\images'
image_paths = glob(os.path.join(image_folder, '*png'))
# mask_paths = ['path/to/mask1.bmp', 'path/to/non_mask2.bmp', ...]
mask_folder = r'C:\Users\user\Desktop\medical_image_analysis\dataset\train\masks'
mask_paths = glob(os.path.join(mask_folder, '*png'))

# Separate minority (non) and majority class data
minority_image_paths = [img for img in image_paths if os.path.basename(img).startswith('Non')]
minority_mask_paths = [msk for msk in mask_paths if os.path.basename(msk).startswith('Non')]

majority_image_paths = [img for img in image_paths if not os.path.basename(img).startswith('Non')]
majority_mask_paths = [msk for msk in mask_paths if not os.path.basename(msk).startswith('Non')]

In [10]:
val_image_folder = r'C:\Users\user\Desktop\medical_image_analysis\dataset\val\images'
val_masks_folder = r'C:\Users\user\Desktop\medical_image_analysis\dataset\val\masks'

val_image_paths = glob(os.path.join(val_image_folder, '*png'))
val_mask_paths = glob(os.path.join(val_masks_folder, '*png'))

In [11]:
train_image_paths = majority_image_paths + minority_image_paths
train_mask_paths = majority_mask_paths + minority_mask_paths

combined = list(zip(train_image_paths, train_mask_paths))
np.random.shuffle(combined)
train_image_paths, train_mask_paths = zip(*combined)
train_image_paths = list(train_image_paths)
train_mask_paths = list(train_mask_paths)

# Create final training dataset with augmentation for minority class
train_dataset = AugmentedDataset(train_image_paths, train_mask_paths, augment=True)

In [12]:

# Validation dataset without augmentation
val_dataset = AugmentedDataset(val_image_paths, val_mask_paths, augment=False)

In [13]:
image, mask = train_dataset.__getitem__(0)
print(image.shape, mask.shape)


(128, 128, 3) (128, 128, 1)


In [14]:
import keras
from keras.layers import Conv2D, Conv2DTranspose,MaxPooling2D,Dropout,Rescaling
# from keras.callbacks import EarlyStop
# from keras.models import Sequential


In [15]:
input_shape = (128,128,3)
inputs = keras.layers.Input(input_shape)

In [16]:
rescaling = Rescaling(1. / 255)(inputs) # rescale input images to [0, 1]

Model Architecture

In [17]:
c1 = keras.layers.Conv2D(16,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(rescaling)
c1 = keras.layers.Dropout(0.1)(c1)
c1 = keras.layers.Conv2D(16,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c1)
p1 = keras.layers.MaxPooling2D(2,2)(c1)

c2 = keras.layers.Conv2D(32,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p1)
c2 = keras.layers.Dropout(0.1)(c2)
c2 = keras.layers.Conv2D(32,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c2)
p2 = keras.layers.MaxPooling2D(2,2)(c2)

c3 = keras.layers.Conv2D(64,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p2)
c3 = keras.layers.Dropout(0.1)(c3)
c3 = keras.layers.Conv2D(64,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c3)
p3 = keras.layers.MaxPooling2D(2,2)(c3)

c4 = keras.layers.Conv2D(128,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p3)
c4 = keras.layers.Dropout(0.2)(c4)
c4 = keras.layers.Conv2D(128,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c4)
p4 = keras.layers.MaxPooling2D(2,2)(c4)

c5 = keras.layers.Conv2D(256,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p4)
c5 = keras.layers.Dropout(0.3)(c5)
c5 = keras.layers.Conv2D(256,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c5)

u6 = keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
u6 = keras.layers.concatenate([u6,c4])
c6 = keras.layers.Conv2D(128,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u6)
c6 = keras.layers.Dropout(0.2)(c6)
c6 = keras.layers.Conv2D(128,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c6)

u7 = keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
u7 = keras.layers.concatenate([u7,c3])
c7 = keras.layers.Conv2D(64,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u7)
c7 = keras.layers.Dropout(0.2)(c7)
c7 = keras.layers.Conv2D(64,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c7)

u8 = keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
u8 = keras.layers.concatenate([u8,c2])
c8 = keras.layers.Conv2D(32,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u8)
c8 = keras.layers.Dropout(0.1)(c8)
c8 = keras.layers.Conv2D(32,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c8)

u9 = keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c8)
u9 = keras.layers.concatenate([u9,c1])
c9 = keras.layers.Conv2D(16,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u9)
c9 = keras.layers.Dropout(0.1)(c9)
c9 = keras.layers.Conv2D(16,(3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c9)

In [18]:
outputs = keras.layers.Conv2D(1, (1,1), activation = 'sigmoid')(c9)

In [19]:
model = keras.Model(inputs = [inputs], outputs = [outputs])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 128, 128, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 16)         448       ['rescaling[0][0]']           
                                                                                                  
 dropout (Dropout)           (None, 128, 128, 16)         0         ['conv2d[0][0]']              
                                                                                              

In [34]:
checkpointer = keras.callbacks.ModelCheckpoint(r'C:\Users\user\Desktop\medical_image_analysis\cxr_model.h5', verbose=1, save_best_only=True)

In [ ]:
# es = keras.callbacks.EarlyStopping()

In [35]:
callbacks = [keras.callbacks.TensorBoard(log_dir = 'mwglog/', histogram_freq=1),checkpointer]

In [28]:
(list(train_dataset))[0]

X_train = [i[0] for i in list(train_dataset)]
X_train_arr = np.array(X_train)

Y_train = [i[1] for i in list(train_dataset)]
Y_train_arr = np.array(Y_train)

print(X_train_arr.shape)
print(Y_train_arr.shape)

(286, 128, 128, 3)
(286, 128, 128, 1)


In [29]:
(list(val_dataset))[0]

X_test = [i[0] for i in list(val_dataset)]
X_test_arr = np.array(X_test)

Y_test = [i[1] for i in list(val_dataset)]
Y_test_arr = np.array(Y_test)

print(X_test_arr.shape)
print(Y_test_arr.shape)

(22, 128, 128, 3)
(22, 128, 128, 1)


In [36]:
results = model.fit(x=X_train_arr, y=Y_train_arr, validation_data = (X_test_arr,Y_test_arr), epochs=35, callbacks=callbacks)

Epoch 1/35
9/9 [==============================] - ETA: 0s - loss: nan - accuracy: 0.7990
Epoch 1: val_loss did not improve from inf
9/9 [==============================] - 31s 3s/step - loss: nan - accuracy: 0.7990 - val_loss: nan - val_accuracy: 0.7924
Epoch 2/35
9/9 [==============================] - ETA: 0s - loss: nan - accuracy: 0.7990
Epoch 2: val_loss did not improve from inf
9/9 [==============================] - 27s 3s/step - loss: nan - accuracy: 0.7990 - val_loss: nan - val_accuracy: 0.7924
Epoch 3/35
9/9 [==============================] - ETA: 0s - loss: nan - accuracy: 0.7990
Epoch 3: val_loss did not improve from inf
9/9 [==============================] - 27s 3s/step - loss: nan - accuracy: 0.7990 - val_loss: nan - val_accuracy: 0.7924
Epoch 4/35
9/9 [==============================] - ETA: 0s - loss: nan - accuracy: 0.7990
Epoch 4: val_loss did not improve from inf
9/9 [==============================] - 26s 3s/step - loss: nan - accuracy: 0.7990 - val_loss: nan - val_accur